# Домашнее задание к лекции «Понятие класса»

In [1]:
import requests

In [2]:
# Задание 1
# Напишите функцию, которая возвращает название валюты (поле ‘Name’)
# с максимальным значением курса с помощью сервиса
# https://www.cbr-xml-daily.ru/daily_json.js

def ex1():
    rate = Rate('full')
    max_rate = 0.0
    max_name = None
    for v in rate.exchange_rates().values():
        if v['Nominal'] != 1:
            continue
        val = v['Value']
        if val > max_rate:
            max_rate, max_name = val, v['Name']
    return max_name


In [3]:
# Задание 2
# Добавьте в класс Rate параметр diff (со значениями True или False),
# который в случае значения True в методах курсов валют (eur, usd итд) будет
# возвращать не курс валюты, а изменение по сравнению в прошлым значением.
# Считайте, self.diff будет принимать значение True только при возврате
# значения курса. При отображении всей информации о валюте он не используется.

def ex2():
    rate = Rate('value', True)
    print('Rate diff:')
    print(' USD: ', rate.usd())
    print(' EUR: ', rate.eur())
    print(' BRL: ', rate.brl())



class Rate:
    def __init__(self, format_='value', diff_='False'):
        self.format = format_
        self.diff = diff_

    def exchange_rates(self):
        """
        Возвращает ответ сервиса с информацией о валютах в виде:

        {
            'AMD': {
                'CharCode': 'AMD',
                'ID': 'R01060',
                'Name': 'Армянских драмов',
                'Nominal': 100,
                'NumCode': '051',
                'Previous': 14.103,
                'Value': 14.0879
                },
            ...
        }
        """
        self.r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')
        return self.r.json()['Valute']

    def make_format(self, currency, diff=False):
        """
        Возвращает информацию о валюте currency в трёх вариантах:
        - полная информация о валюте при self.format = 'full':
        Rate('full').make_format('EUR')
        {
            'CharCode': 'EUR',
            'ID': 'R01239',
            'Name': 'Евро',
            'Nominal': 1,
            'NumCode': '978',
            'Previous': 79.6765,
            'Value': 79.4966
        }

        Rate('value').make_format('EUR')
        79.4966

        Rate('value').make_format('EUR', diff=True)

        """
        response = self.exchange_rates()

        if currency in response:
            if self.format == 'full':
                return response[currency]
            
            if self.format == 'value':
                res = response[currency]['Value']
                if diff:
                    res = round(res - response[currency]['Previous'], 4)
                return res
        return 'Error'

    def eur(self):
        """Возвращает курс евро на сегодня в формате self.format
        или изменение по сравнению в прошлым значением если self.diff==True"""
        return self.make_format('EUR', self.diff)

    def usd(self):
        """Возвращает курс доллара на сегодня в формате self.format
        или изменение по сравнению в прошлым значением если self.diff==True"""
        return self.make_format('USD', self.diff)

    def brl(self):
        """Возвращает курс бразильского реала на сегодня в формате self.format
        или изменение по сравнению в прошлым значением если self.diff==True"""
        return self.make_format('BRL', self.diff)

In [9]:
from klass import ex2
ex2()

Rate diff:
 USD:  -0.3419
 EUR:  -0.0054
 BRL:  0.1101


In [4]:
# Задание 3
# Напишите класс Designer, который учитывает количество международных премий.
# Подсказки в коде занятия в разделе “Домашнее задание задача 3”.

class Employee:
    def __init__(self, name, seniority):
        self.name = name
        self.seniority = seniority
        self.grade = 1

    def grade_up(self):
        """Повышает уровень сотрудника"""
        self.grade += 1

    def publish_grade(self):
        """Публикация результатов аккредитации сотрудников"""
        print(self.name, self.grade)

    def check_if_it_is_time_for_upgrade(self):
        pass


class Designer(Employee):
    def __init__(self, name, seniority):
        # По умолчанию при выходе на работу сотрудник уже имеет две премии,
        # т.е. по два бала за премию получаем +4 к seniority
        super().__init__(name, seniority)
        self.awards = 2

    def _check_grade_up(self):
        # условие повышения на 1 грейд за каждые 7 баллов
        # для каждой международной премии +2 бала
        if (self.awards * 2 + self.seniority) % 7 == 0:
            self.grade_up()
        
        # публикация результатов
        return self.publish_grade()

    def awarding(self):
        # Присуждение новой международной премии
        self.awards += 1
        
        return self._check_grade_up()


    def check_if_it_is_time_for_upgrade(self):
        # для каждой аккредитации увеличиваем счетчик на 1
        self.seniority += 1

        return self._check_grade_up()
